# Rationale

ACS for MAB v1 was wayyy unsophisticated.

New idea here is to have a directed (asymmetric TSP) graph, where each city has edges leading away from it that each represents an arm. Hence, for a k-arm MAB, we have k+1 cities.

We also have to make sure that each set of same-$\theta$-ed edges form a tour, such that the shortest tour indicates the shortest arm. i.e. $k$ tours out of a possible $(k-1)!$ actually represent arms.

Denote the true thetas $\left\{ \theta_i = (\alpha_i, \beta_i) \right\}_{i=1}^k$. For purposes of experimental simplicity, let $\alpha_i \sim \mathcal{U}(1,2)$ and $\beta_i \sim \mathcal{U}(1,2)$.

The cost for each edge leading out from an arbitrary node $u$ would be given by
$$
\delta(u,v_i) = \frac{\alpha_i}{\beta_i} \\
$$
(note that the $i$ in the above expression is actually based on which $\theta_i$ corresponds to which end node $v_i$, not the other way around)

Then, using a $\Gamma$ distribution
$$
\mathbb{E} [\eta (u, v_i)] = \frac{\alpha_i}{\beta_i} \\
$$
such that
$$
\eta (u, v_i) \sim \Gamma ( \alpha_i, \beta_i )
$$

Above it, we just have Gamma bandits. Choose the edge $\implies$ pull the arm. Now we have a valid way to quantify regret.


# Les Codez

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
plt.style.use(['ggplot', 'seaborn-poster'])
from tqdm import tqdm

## MAB

In [3]:
ARMS = 10

## ACS

### Initialize

In [4]:
NODES = ARMS + 1
ANTS = 100
ITERATIONS = 1000
Q_0 = 0.0
RHO = 0.1
ALPHA = 1e-4
BETA = 10

In [5]:
TRUE_ALPHAS = np.random.uniform(low=1, high=2, size=ARMS)
# TRUE_BETAS = np.random.uniform(low=1, high=2, size=ARMS)
TRUE_BETAS = np.sqrt( TRUE_ALPHAS / 0.01 )

In [6]:
def generate_arm_tours(k):
    """
    which edge corresponds to which arm?
    k: number of arms
    k+1: number of cities
    """
    A = -np.eye(k+1)
    for i in xrange(k+1):
        np.put(A[i,:], xrange(i+1, i+1+k), xrange(0,k), mode='wrap')
    return A.astype(int)

In [7]:
def tour_length_nn():
    """
    find approx tour length using nn heuristic
    adapted for our scenario where we can only observe stochastic cost
    """
    unvisited_nodes = set(xrange(NODES))
    current_node = np.random.choice(xrange(NODES))
    unvisited_nodes -= set([current_node])
    total_cost = 0
    for step in xrange(NODES-1):
        current_arms = A[current_node,:]
        costs = np.array([np.random.gamma(TRUE_ALPHAS[i], \
                                 1.0/TRUE_BETAS[i]) \
                          if i != -1 else np.inf \
                          for i in current_arms])
        min_cost = costs[list(unvisited_nodes)].min()
        min_arm = current_arms[np.where(costs == min_cost)]
        total_cost += min_cost
        current_node = np.where(current_arms == min_arm)[0][0]
    return total_cost

In [8]:
def generate_starts():
    return np.random.choice(xrange(NODES), size=ANTS)

In [9]:
def generate_tour_tracker():
    Ts = -np.ones([ANTS, NODES])
    starts = generate_starts()
    Ts[xrange(ANTS), starts] = 0
    return Ts.astype(int)

In [10]:
def reset_tour_lengths():
    return np.zeros(ANTS)

In [11]:
def eta(a,b):
    arm = A[a,b]
    assert arm != -1, 'You done goofed. You returned to the same node.'
    return np.random.gamma(TRUE_ALPHAS[arm], 1.0/TRUE_BETAS[arm])

In [12]:
def eta_psi(a,b):
    """
    We need eta for calculating tour length (i.e. reward)
    but it's stochastic so we need to pass it back to the ant
    """
    cost = eta(a,b)
    return cost, taus[a,b] * ( cost ** BETA )

In [13]:
def local_update_taus(a,b):
    taus[a,b] = (1 - RHO) * taus[a,b] + RHO * TAU_0

In [14]:
def take_step(ant):
    q = np.random.uniform()
    current_node = Ts[ant,:].argmax()
    unvisited_nodes = np.where(Ts[ant,:] == -1)[0]
    eta_psis = np.array([eta_psi(current_node, unvisited_node) \
                    for unvisited_node in unvisited_nodes])
    if q <= Q_0: # exploit
        next_node = unvisited_nodes[ eta_psis[:,1].argmax() ]
        eta = eta_psis[ eta_psis[:,1].argmax(), 0 ]
    else: # explore
        ps = eta_psis[:,1] / eta_psis[:,1].sum()
        next_node = np.random.choice(unvisited_nodes, p=ps)
        eta = eta_psis[ np.where(unvisited_nodes == next_node), 0 ]
    tour_lengths[ant] += eta
    current_step = Ts[ant,:].max()
    Ts[ant,next_node] = current_step + 1
    local_update_taus(current_node,next_node)

In [15]:
def update_global_best():
    global global_best_tour_lengths
    global global_best_tour

    best_tour_length = tour_lengths.min()
    if best_tour_length < global_best_tour_lengths[-1]:
        best_ant = tour_lengths.argmin()
        best_tour = Ts[best_ant,:]
        global_best_tour = np.array( sorted( zip( best_tour, xrange(len(best_tour))), \
                                       key=lambda x: x[0])).T[1]
        global_best_tour_lengths = np.append(global_best_tour_lengths, best_tour_length)

In [16]:
def global_update_taus():
    global taus
    global global_best_tour_lengths
    taus = (1 - ALPHA) * taus
    for i in xrange(len(global_best_tour) - 1):
        a = global_best_tour[i]
        b = global_best_tour[i+1]
        taus[a,b] += ALPHA / global_best_tour_lengths[-1]
    taus[global_best_tour[-1], global_best_tour[0]] += ALPHA / global_best_tour_lengths[-1]

### Run it!

In [17]:
A = generate_arm_tours(ARMS)

In [18]:
TAU_0 = 1.0 / ( NODES * tour_length_nn() )

In [19]:
global_best_tour = np.zeros(NODES)
global_best_tour_lengths = np.array([np.inf])
taus = TAU_0 * np.ones(A.shape)

In [20]:
for i in tqdm(xrange(ITERATIONS)):
    Ts = generate_tour_tracker()
    tour_lengths = reset_tour_lengths()
    for step in xrange(NODES - 1):
        for ant in xrange(ANTS):
            take_step(ant)
    update_global_best()
    global_update_taus()

100%|██████████| 1000/1000 [01:19<00:00, 12.60it/s]


In [21]:
global_best_tour_lengths

array([        inf,  1.61875459,  1.39702746,  1.38089239,  1.33967161,
        1.28252765])

In [22]:
global_best_tour

array([ 9,  3,  6,  4,  5,  2,  1, 10,  0,  7,  8])

In [23]:
# expected length of shortest tour
(TRUE_ALPHAS / TRUE_BETAS).min() * ARMS

1.0333573795719644

In [24]:
# expected length of an average tour
(TRUE_ALPHAS / TRUE_BETAS).mean() * ARMS

1.2240002281954943

In [25]:
regret = 0
best_edges = np.where(A==(TRUE_ALPHAS / TRUE_BETAS).argmin())
for i in xrange(ARMS):
    a = global_best_tour[i]
    b = global_best_tour[i+1]
    if best_edges[1][a] != b:
        regret += 1
a = global_best_tour[-1]
b = global_best_tour[0]
if best_edges[1][a] != b:
    regret += 1

In [26]:
regret

10